In [ ]:
"""
Time Corrleation Analysis of Gravitational Waves and Gamma Ray Bursts

Import package of dealing with Fermi data
"""
from gw_grb_correlation.Fermi.util import duration, filtering, read_GW_data
from gw_grb_correlation.Fermi.data_preprocessing import create_dataframe_and_name_column_from_data_files


"""
Load Fermi data
Be sure that you already run the fermi_data_exploration.ipynb notebook to download and process the Fermi data
"""
fermi_data = create_dataframe_and_name_column_from_data_files(data_type="fermi")

"""
Filter out short GRB data
"""
short_GRB_data = filtering(fermi_data, criteria={'T90': lambda x: x <= 2.1})

"""
Calculate the duration (difference between max TSTOP and min TSTART) and count the short GRB event number
"""
duration_short_GRB = duration(short_GRB_data)
event_num_short_GRB = len(short_GRB_data)

"""
Calculate the average occurrence rate of short GRB (events per unit of time)
"""
average_occurrence_rate = event_num_short_GRB / duration_short_GRB

"""
Print the results
"""
print(f"Number of short GRB: {event_num_short_GRB}")
print(f"Total duration: {duration_short_GRB} seconds")
print(f"Average occurrence rate: {average_occurrence_rate} short GRB per second")

In [ ]:
from gw_grb_correlation.Fermi.util import read_GW_data, remove_duplicate_times_in_gw_data, compare_time_within_range
"""
Load GW data
To run the time correlation analysis between GRB and GW, be sure that you already run the GW data preprocessing notebook first to get the totalgwdata.csv file and places it in the ./gw_data/ directory.
"""
gw_data = read_GW_data(f"./gw_data/totalgwdata.csv")
gw_times = remove_duplicate_times_in_gw_data(gw_data)

"""
Find matched GRB-GW event pairs
"""
match = compare_time_within_range(short_GRB_data, gw_times, time_range_seconds=86400*3)
filtered_gw_events = gw_data[gw_data['times'].isin(match['gw_time'])]

"""
Save the matched GRB-GW event pairs and filtered GW events to CSV files
"""
match.to_csv("GRB_GW_event_pairs.csv", index=False)
filtered_gw_events.to_csv("Filtered_GW_events.csv", index=False)

In [ ]:
"""
Angular Correlation Analysis of Gravitational Waves and Gamma Ray Bursts
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from skimage import measure
from shapely.geometry import Point, Polygon
from scipy import signal

#Read both the gravitational wave data and the reduced fermi data

grbdata=pd.read_csv('./GRB_GW_event_pairs.csv')
gwdata=pd.read_csv('./gw_data/totalgwdata.csv')

#create a smoothing guassian kernel used for creating contours

def getgausskern(sigma,n):
   array=np.zeros([n,n])
   for ii in range(n):
     for jj in range(n):
       array[ii,jj]=np.exp(-((ii+(1-n)/2)**2+(jj+(1-n)/2)**2)/(2*sigma**2))
   return array

gauskern=getgausskern(3,25)

#Create angle for the bins that will be used for GW data

angle=np.pi/180

#Create List that will store if any points lie within contours

within=[]

#Create a list that will contain the area of the GW contours

where=[]


#Create a 2D histogram of gravitational waves that are time correlated with GRBs.
#This goes through all gw to find where they are time correlated. Then bins them
#Into a sky map.

for ll in range(len(grbdata['gw_time'])):
  print(ll)
  time=grbdata['gw_time'][ll]
  #timedif=grbdata['time_diff'][ii]+2000
  timedif=60
  radecarray=np.zeros([360,180])
  dif=np.abs(gwdata['times']-time)
  indices=np.where(dif <= timedif)[0]
  #print(len(indices))

  #indices=np.where(gwdata['times']==time)[0]
  #print(len(indices))

  ras=gwdata['ra'][min(indices):max(indices)+1]
  decs=gwdata['dec'][min(indices):max(indices)+1]
  weights=gwdata['weights'][min(indices):max(indices)+1]
  weigh=np.array(weights)
  #print(len(weigh))
  #print(weigh)
  #print(ras)
  #print(decs)
  #print(weights)

  for ii in range(360):
   radif=np.abs(ras-ii*angle)
   raindexs=np.where(radif <= angle)[0]
   if len(raindexs)==0:
    continue
   else:
    for jj in range(180):
      decdif=np.abs(decs+np.pi/2-jj*angle)
      decindexs=np.where(decdif <= angle)[0]
      if len(decindexs)==0:
        continue
      else:
       indexs=np.intersect1d(raindexs,decindexs)
       #print(indexs)
       if len(indexs)==0:
         continue
       else:
        for k in indexs:
         #print(k)
         #print(weigh[k])
         radecarray[ii,jj]+=weigh[k]
  if np.amax(radecarray) == 0:
    print('Something wrong with time ', time)
  else:
   #show the GW histogram
   fig, ax = plt.subplots()
   ax.imshow(radecarray)
   ax.scatter([grbdata.iloc[int(ll),1:3]['grb_dec']+90],[grbdata.iloc[int(ll),1:3]['grb_ra']])
   ax.axis('image')
   ax.set_xlabel("Declination")
   ax.set_ylabel("Right Ascension")
   plt.show()

  #Next we convole the GW hist with the kernel to smooth the bins. This allows for contours to be created
    #that work well with the pip algorithym. 
    
  convolved=signal.fftconvolve(radecarray,gauskern,mode='same')
  #plt.imshow(convolved)
  #plt.show()
  print(np.amax(convolved))

#Append to where the area of the contours
    
  where.append(len(np.where(convolved>np.amax(convolved)*0.1)[0]))

#create contours
    
  contours = measure.find_contours(convolved, np.amax(convolved)*0.1)
    
  # Display the image and plot all contours found
    
  fig, ax = plt.subplots()
  ax.imshow(convolved, cmap=plt.cm.gray)
    
  for contour in contours:
    ax.plot(contour[:, 1], contour[:, 0], linewidth=2)

  ax.scatter([grbdata.iloc[int(ll),1:3]['grb_dec']+90],[grbdata.iloc[int(ll),1:3]['grb_ra']],color='r')

  ax.axis('image')
  ax.set_xlabel("Declination")
  ax.set_ylabel("Right Ascension")
  plt.show()

#Create a point that is the correct GRB location
    
  point=Point(grbdata.iloc[int(ll),1:3]['grb_ra'],grbdata.iloc[int(ll),1:3]['grb_dec']+90)

#Run pip to determine if the GRB lies inside any contour. If it lies in any of them, within will then say True
    
  iftrue=False
  for coonts in contours:
   poly = Polygon(coonts)

  # PIP test with 'contains'
   print(poly.contains(point))
   if poly.contains(point)==True:
    iftrue=True
  if iftrue==True:
    within.append(True)
  else:
    within.append(False)

#Print how many true and falses there were

print(within)

#Calculate average area of the GW contours then print the percentage

averagearea=np.mean(where)
print(averagearea/(360*180))


In [ ]:
"""
Angular Correlation Analysis of Gravitational Waves and Gamma Ray Bursts using the Neural Network predicted GRB locations
This code is similar to the above, but uses the predicted GRB locations from the neural network instead of the actual GRB locations.

Be sure to run the neural network code fermi_position_predictor.ipynb first to get the predicted GRB locations and save them in the GRB_GW_event_pairs_predict.csv file.
"""
#Read both the gravitational wave data and the reduced fermi data

grbdata=pd.read_csv('./GRB_GW_event_pairs_predict.csv')
gwdata=pd.read_csv('./gw_data/totalgwdata.csv')

#create a smoothing guassian kernel used for creating contours

def getgausskern(sigma,n):
   array=np.zeros([n,n])
   for ii in range(n):
     for jj in range(n):
       array[ii,jj]=np.exp(-((ii+(1-n)/2)**2+(jj+(1-n)/2)**2)/(2*sigma**2))
   return array

gauskern=getgausskern(3,25)

#Create angle for the bins that will be used for GW data

angle=np.pi/180

#Create List that will store if any points lie within contours

within=[]

#Create a list that will contain the area of the GW contours

where=[]


#Create a 2D histogram of gravitational waves that are time correlated with GRBs.
#This goes through all gw to find where they are time correlated. Then bins them
#Into a sky map.

for ll in range(len(grbdata['gw_time'])):
  print(ll)
  time=grbdata['gw_time'][ll]
  #timedif=grbdata['time_diff'][ii]+2000
  timedif=60
  radecarray=np.zeros([360,180])
  dif=np.abs(gwdata['times']-time)
  indices=np.where(dif <= timedif)[0]
  #print(len(indices))

  #indices=np.where(gwdata['times']==time)[0]
  #print(len(indices))

  ras=gwdata['ra'][min(indices):max(indices)+1]
  decs=gwdata['dec'][min(indices):max(indices)+1]
  weights=gwdata['weights'][min(indices):max(indices)+1]
  weigh=np.array(weights)
  #print(len(weigh))
  #print(weigh)
  #print(ras)
  #print(decs)
  #print(weights)

  for ii in range(360):
   radif=np.abs(ras-ii*angle)
   raindexs=np.where(radif <= angle)[0]
   if len(raindexs)==0:
    continue
   else:
    for jj in range(180):
      decdif=np.abs(decs+np.pi/2-jj*angle)
      decindexs=np.where(decdif <= angle)[0]
      if len(decindexs)==0:
        continue
      else:
       indexs=np.intersect1d(raindexs,decindexs)
       #print(indexs)
       if len(indexs)==0:
         continue
       else:
        for k in indexs:
         #print(k)
         #print(weigh[k])
         radecarray[ii,jj]+=weigh[k]
  if np.amax(radecarray) == 0:
    print('Something wrong with time ', time)
  else:
   #show the GW histogram
   fig, ax = plt.subplots()
   ax.imshow(radecarray)
   ax.scatter([grbdata.iloc[int(ll),1:3]['grb_dec']+90],[grbdata.iloc[int(ll),1:3]['grb_ra']])
   ax.axis('image')
   ax.set_xlabel("Declination")
   ax.set_ylabel("Right Ascension")
   plt.show()

  #Next we convole the GW hist with the kernel to smooth the bins. This allows for contours to be created
    #that work well with the pip algorithym. 
    
  convolved=signal.fftconvolve(radecarray,gauskern,mode='same')
  #plt.imshow(convolved)
  #plt.show()
  print(np.amax(convolved))

#Append to where the area of the contours
    
  where.append(len(np.where(convolved>np.amax(convolved)*0.1)[0]))

#create contours
    
  contours = measure.find_contours(convolved, np.amax(convolved)*0.1)
    
  # Display the image and plot all contours found
    
  fig, ax = plt.subplots()
  ax.imshow(convolved, cmap=plt.cm.gray)
    
  for contour in contours:
    ax.plot(contour[:, 1], contour[:, 0], linewidth=2)

  ax.scatter([grbdata.iloc[int(ll),1:3]['grb_dec']+90],[grbdata.iloc[int(ll),1:3]['grb_ra']],color='r')

  ax.axis('image')
  ax.set_xlabel("Declination")
  ax.set_ylabel("Right Ascension")
  plt.show()

#Create a point that is the correct GRB location
    
  point=Point(grbdata.iloc[int(ll),1:3]['grb_ra'],grbdata.iloc[int(ll),1:3]['grb_dec']+90)

#Run pip to determine if the GRB lies inside any contour. If it lies in any of them, within will then say True
    
  iftrue=False
  for coonts in contours:
   poly = Polygon(coonts)

  # PIP test with 'contains'
   print(poly.contains(point))
   if poly.contains(point)==True:
    iftrue=True
  if iftrue==True:
    within.append(True)
  else:
    within.append(False)

#Print how many true and falses there were

print(within)

#Calculate average area of the GW contours then print the percentage

averagearea=np.mean(where)
print(averagearea/(360*180))